# Notebook : NewDustSpot

Goal :  This Notebook is used to identify new dust spots that appeared on sensor since vendor data 

Author : P.Antilogus 

Version : 7th July 2019 21:00 

- It uses superfats at 500 nm taken at various steps of the raft-focal plane construction , and looks for new attenuated pixels.
- The different deffects/attenuated pixels are linked by a simple but robust topological algo to build a "dust spot" in 2D .
- A vignet associated to each dust spot is ploted with its absolute location on sensor and with an information of its assocaited extinction . 
- by default only spots with one pixel wit a transmission bellow 0.95 are listed . 

Remarks :
- The usual way to use this script is to search for new dust spots compared to sensor vendor data . Only new dust spots , not present in vendor
   data, can be identified/ploted. Still some coating defect intensity are highlt dependent of the light source, and may appear as new dust   spot , even if already present in vendor data.
- A few sensor defect also appear as false positive , futher cleaning are certainly possible . 
- we could easily provide a 2D raft image with all new dust spots , to allow a direct visual inspection / comparison with the raft . A first step in this direction has been to produce a pickel file with all the information associated to each dust spot , furhter developement on the content of this pickel file could allow to prepare nice summary display. (see last boxes of this notebook)  

In [ ]:
import glob
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import numpy as np
import pickle

Configuration : 
 - a list of raft(s) has to be provided  ==> raft_list 
 - select the # data set "function" ==> data_id (see in following box for config ) : 
     - select the image to be used for reference ( will look for new dusts compared to this dataset) , default : vendor data
     - select the image to look for new dust , default TS3 data ( but last acquire raft data should be a good option ) 
     - there is 2 other set of data used for complementory information (raft data taken in # condition ) 
     - example :
         - data_id=[0,1,2,3] : reference data = vendor, look for new dust = TS3 data , auxilary data ( for plots only ) = 2 (RTM BNL ) and 3 ( RTM SLAC ) 
         - data_id=[0,3,1,2] : reference data = vendor, look for new dust = SLAC RTM , auxilary data ( for plots only ) = 1 (TS3 data if any) and 2  (RTM BNL )
         - after CCD cleaning we could use "new raft" as a reference and looks for dust in "old raft" to see how many were removed ... To see how many are left , we do the usual comparison with vendor data. 

     

In [ ]:
# CONFIGURATION FOR THE CURRENT EXECUTION  ========================================
# ---- raft and associated run ============ To be updated if needed 
# if no run are given (=*) , the last run taken will be used 
raft_list=[{'RTM':'RTM-004','Dust1_lab':'BNL','Dust1_run':'*','Dust2_lab':'SLAC','Dust2_run':'10141'}]
#raft_list=[{'RTM':'RTM-012','Dust1_lab':'SLAC','Dust1_run':'11062','Dust2_lab':'SLAC','Dust2_run':'11067'}]
# ---- configuration for the dust search and visualisation 
# data set id , define a number (0,1,2,3) associated to a type of data , this is fixed for ever  
# 0 = 'Vendor'
# 1 = 'TS3'
# 2 = 'Dust1'
# 3 = 'Dust2'  
# then in data_id we select the # dataset for reference , dust search and comparison :
# for example : data_id=[0,2,1,3] corresponds to :
#                 0 ==> the reference (=minimal dust) , is from vendor data 
#                 2 ==> the data where we are looking for dust  (= in general last data set ) , here is Dust1 data as specificed in raft_list
#                 1 ==> intermediate data set , is from TS3 
#                 3 ==> the other intermediate data set is from Dust2 data as specificed in raft_list 
data_id=[0,2,1,3]


In [ ]:
count =!set | grep  .ncsa.illinois.edu | wc
if int(count[0].split()[0])> 1 : 
    NCSA=True
else : 
    NCSA=False
if NCSA : 
    SLACmirror='/project/rgruendl/SLACmirror'
    # the list of super flat for a raft - run : BNL_RAFT_ROOT+raft+'/'+run+'/cte_raft/*/*/'+'Sxx'+'/*_superflat_high.fits' 
    BNL_RAFT_ROOT=SLACmirror+'/BNLmirror/mirror/BNL-prod/prod/LCA-11021_RTM/LCA-11021_'
    #
    #SLAC_RAFT_ROOT=SLACmirror+'/SLACgpfs/jh_archive/LCA-11021_RTM/LCA-11021_'
    #
    # the list of TS3 BNL superflat for a CCD will be BNL_TS3_ROOT +CCD_NAME+'/*/cte*/v0/*/'+CCD_NAME+'_superflat_high.fits'
    BNL_TS3_ROOT=SLACmirror+'/BNLmirror/mirror/BNL-prod/prod/*-CCD/'
    #
    # the list of TS3 vendor super flat for a CCD will be VENDOR_ROOT +CCD_NAME+'/*/cte*/v0/*/'+CCD_NAME+'_superflat_high.fits'
    VENDOR_ROOT=SLACmirror+'/jobHarness/jh_archive/*-CCD/'
    #
    SLAC_RAFT_ROOT=SLACmirror+'/fs3/jh_archive/LCA-11021_RTM/LCA-11021_'
else :
    # the list of super flat for a raft - run : BNL_RAFT_ROOT+raft+'/'+run+'/cte_raft/*/*/'+'Sxx'+'/*_superflat_high.fits' 
    BNL_RAFT_ROOT='/nfs/farm/g/lsst/u1/mirror/BNL-prod/prod/LCA-11021_RTM/LCA-11021_'
    #
    SLAC_RAFT_ROOT='/gpfs/slac/lsst/fs1/g/data/jobHarness/jh_archive/LCA-11021_RTM/LCA-11021_'
    #
    # the list of TS3 BNL superflat for a CCD will be BNL_TS3_ROOT +CCD_NAME+'/*/cte*/v0/*/'+CCD_NAME+'_superflat_high.fits'
    BNL_TS3_ROOT='/nfs/farm/g/lsst/u1/mirror/BNL-prod/prod/*-CCD/'
    #
    # the list of TS3 vendor super flat for a CCD will be VENDOR_ROOT +CCD_NAME+'/*/cte*/v0/*/'+CCD_NAME+'_superflat_high.fits'
    VENDOR_ROOT='/nfs/farm/g/lsst/u1/jobHarness/jh_archive/*-CCD/'
    #



The following box contain a set of subroutines that makes this notbook self consistent ( no outside software needed)

In [ ]:
def size_image(image) :
    # extract the overscan location to be used in python table  from the DATASEC keyword
    r=image[1].header['DATASEC'][1:-1].split(',')
    x=r[0].split(':')
    y=r[1].split(':')
    return int(x[0])-1,int(x[1]),int(y[0])-1,int(y[1])
#
def cor_image(image_fits,label='',print_run=True,plot=False) :
    # This routine with overscan 2D  correct the image , and normalize to the median flux 
    # input : image = 2D fits image ,  plot : flag to do the control plots  
    # print('process superflat ',image_fits)
    # Number of line and column droped at the start and end of image to compute the normalisation of the image 
    drop=25
    # open it
    image=pyfits.open(image_fits)
    # do we print information on file 
    if print_run : 
        try : 
            date_obs=image[0].header['DATE-OBS']
        except :
            date_obs='unknown'
        try : 
            run=image[0].header['RUNNUM']
        except :
            run='unknown'
        print(label,' date of observation= ',date_obs,' run= ',run)
    # image shape 
    start_col,end_col,start_line,end_line=size_image(image)
    ny,nx=np.shape(image[1].data)
    im_cor=np.zeros((16,ny,nx))
    # offset to get to an overscan pixel not distrubed by image area : 2 pixels should do most of the job 
    offset=2
    #  first oversca pixel considred to compute bias 
    last_col=end_col+offset
    #
    last_line=end_line+offset
    #
    for im in range(1,17) :
        im0=im-1
        # correction : remark we are not taking a median ( not precise enough) ...so we can have issue 
        # due to deffect in particular in the // direction ( bleeding line ...)  ==> action 
        # correction per column to the average correction at serial overscan
        col_cor=image[im].data[last_line:,:].mean(axis=0)-image[im].data[last_line:,last_col:].mean()
        # correction per line function of the overscan
        line_cor=image[im].data[:,last_col:].mean(axis=1)
        # immage corrected 
        for i in range(ny) :
            im_cor[im0,i,:]=image[im].data[i,:]-line_cor[i]
        for j in range(nx):
            im_cor[im0,:,j]-=col_cor[j]
        # nomralize to the <flux>
        ref=np.median(im_cor[im0,drop:-drop,drop:-drop])
        im_cor[im0,:,:]=im_cor[im0,:,:]/ref
        if plot : 
            fig=plt.figure(figsize=[10,10])
            fig.suptitle(im)
            ax = fig.add_subplot(2,3,1)
            plt.plot(im_cor[im0,start_line:end_line,start_col:end_col].mean(axis=0))
            ax = fig.add_subplot(2,3,4)
            plt.plot(im_cor[im0,start_line:end_line,start_col:end_col].mean(axis=1))
            ax = fig.add_subplot(2,3,2)
            plt.plot(im_cor[im0,last_line:,start_col:end_col].mean(axis=0))
            ax = fig.add_subplot(2,3,5)
            plt.plot(im_cor[im0,end_line:,last_col:].mean(axis=1))
            ax = fig.add_subplot(2,3,3)
            plt.plot(col_cor[:])
            ax = fig.add_subplot(2,3,6)
            plt.plot(line_cor[:])

            plt.show()
    return im_cor[:,start_line:end_line,start_col:end_col]
#
def found_and_plot_dust_spot(ccd,data_ref,data_dust,data_ext1,data_ext2,label,cut=0.15,line_cut=3):
    # data_ref  :   image of reference , we will look for new dust since data_ref , in general it's vendor data
    # data_dust :   image used to look for new dust 
    # data_extra1 and data_extra2 :" the area associated to the detected dust will be also ploted for these images
    # cut       :   in case of dipole  in the dust image 0.15 should be used , else 0.05 ? 
    # cut_line  : number of line at start and end of image not considere for dust ( edge effect + bad data = lots of fake )
    spot_list=np.zeros((16),dtype=np.object_)
    dust_image=np.zeros((16),dtype=np.object_)
    # 
    # image size in y and x 
    ylen=len(data_ref[0,:,0])
    xlen=len(data_ref[0,0,:])
    # Loop on amplifier
    for im in range(16) :
        #
        dif_im=data_ref[im,:,:]-data_dust[im,:,:]
        #
        print("=================================================================================================")
        print('Summary for amplifier ',im+1,' for ccd ',ccd)
        print('')
        #
        # the "line_cut" first and last line(s) should be droped due to edge effect associated probably to voltage 
        dust=np.array([(i,j) for i in range(line_cut,ylen-line_cut) for j in range(xlen) if dif_im[i,j]>cut],dtype=int)
        print('we found ',len(dust),' occulted pixels in dusty image (',label[1],') not present in reference (',label[0],')')
        print('')
        print('')
        spot_list[im]=[]
        dust_image[im]=[]
        spot_coordinate=[]
        defect_list=np.zeros((2))
        if len(dust)>0 :
            # look for dust : topological agreagation : along line and then along column 
            min_dust=dust[0]
            dust_start=0
            nb_dust=len(dust)
            #
            for idust in range(1,nb_dust) :
                dust_cur=dust[idust]
                if dust_cur[0]>min_dust[0]+1 :
                    # end of topological same or line+1 connection 
                    # look for topological connection along column , so first re-order the dust in function of
                    # the column first , line second ....
                    dust_index=np.lexsort((dust[dust_start:idust,0],dust[dust_start:idust,1]))
                    #
                    xmin=dust[dust_start+dust_index[0]][1]
                    ymin=dust[dust_start+dust_index[0]][0]
                    xmax=xmin
                    ymax=ymin
                    minc_dust=dust[dust_start+dust_index[0]]
                    size=1
                    transp=data_dust[im,ymin,xmin]
                    for jdust in range(1,len(dust_index)) :
                        dustc_cur=dust[dust_index[jdust]+dust_start]
                        if dustc_cur[1]>minc_dust[1]+1 :
                            spot_coordinate.append([ymin,ymax,xmin,xmax,size,transp/size])
                            xmin=dustc_cur[1]
                            ymin=dustc_cur[0]
                            xmax=xmin
                            ymax=ymin
                            size=1
                            transp=data_dust[im,dustc_cur[0],dustc_cur[1]]
                        else :
                            xmin=min(dustc_cur[1],xmin)
                            ymin=min(dustc_cur[0],ymin)
                            xmax=max(dustc_cur[1],xmax)
                            ymax=max(dustc_cur[0],ymax) 
                            size+=1
                            transp+=data_dust[im,dustc_cur[0],dustc_cur[1]]
                        minc_dust=dustc_cur
                    # save the last dust 
                    spot_coordinate.append([ymin,ymax,xmin,xmax,size,transp/size])
                    # and the new dust index
                    dust_start=idust
                # 
                min_dust=dust_cur
            # idem for the last dust
            # the column first , line second ....
            dust_index=np.lexsort((dust[dust_start:nb_dust,0],dust[dust_start:nb_dust,1]))
            #
            xmin=dust[dust_start+dust_index[0]][1]
            ymin=dust[dust_start+dust_index[0]][0]
            xmax=xmin
            ymax=ymin
            minc_dust=dust[dust_start+dust_index[0]]
            size=1
            transp=data_dust[im,ymin,xmin]
            for jdust in range(1,len(dust_index)) :
                dustc_cur=dust[dust_index[jdust]+dust_start]
                if dustc_cur[1]>minc_dust[1]+1 :
                    spot_coordinate.append([ymin,ymax,xmin,xmax,size,transp/size])
                    xmin=dustc_cur[1]
                    ymin=dustc_cur[0]
                    xmax=xmin
                    ymax=ymin
                    size=1
                    transp=data_dust[im,dustc_cur[0],dustc_cur[1]]
                else :
                    xmin=min(dustc_cur[1],xmin)
                    ymin=min(dustc_cur[0],ymin)
                    xmax=max(dustc_cur[1],xmax)
                    ymax=max(dustc_cur[0],ymax) 
                    size+=1
                    transp+=data_dust[im,dustc_cur[0],dustc_cur[1]]
                minc_dust=dustc_cur
            # save the last dust 
            spot_coordinate.append([ymin,ymax,xmin,xmax,size,transp/size])
            #  loop on identified dust spot  
            for cur_spot in spot_coordinate :
                xmin=max(0,cur_spot[2]-10)
                xmax=min(xlen,cur_spot[3]+10)
                ymin=max(0,cur_spot[0]-10)
                ymax=min(ylen,cur_spot[1]+10)
                transparency=np.min(data_dust[im,cur_spot[0]:cur_spot[1]+1,cur_spot[2]:cur_spot[3]+1])
                # Spot selection : ask for a transparency less than 0.95 for the min transparency and for a positive transparency 
                if transparency<0 or transparency>0.95 : continue
                fig=plt.figure(figsize=[15,5])
                spot_list[im].append(cur_spot)
                dust_image[im].append([ymin,xmin,data_dust[im,ymin:ymax,xmin:xmax]])
                if transparency < .6 :
                            defect_list[1]+=1
                else :
                            defect_list[0]+=1
                xlabel='ampli %d dust x=%d:%d y=%d:%d minimun transparency(new TS3)=%4.2f ' % (im+1, xmin+1,xmax+1,ymin+1,ymax+1,transparency)
                #fig.suptitle(xlabel)
                ax = fig.add_subplot(1,4,1)
                ax.set_title('ref='+label[0])
                vmin=min(np.min(data_ref[im,ymin:ymax,xmin:xmax]),np.min(data_dust[im,ymin:ymax,xmin:xmax]))
                vmax=min(np.max(data_ref[im,ymin:ymax,xmin:xmax]),np.max(data_dust[im,ymin:ymax,xmin:xmax]))
                plt.imshow(data_ref[im,ymin:ymax,xmin:xmax],extent=[xmin,xmax,ymin,ymax],vmin=vmin,vmax=vmax)
                if len(data_ext1) > 0  :
                    ax = fig.add_subplot(1,4,2)
                    ax.set_title(label[2])
                    plt.imshow(data_ext1[im,ymin:ymax,xmin:xmax],extent=[xmin,xmax,ymin,ymax],vmin=vmin,vmax=vmax)
                if len(data_ext2) > 0  :
                    ax = fig.add_subplot(1,4,3)
                    ax.set_title(label[3])
                    plt.imshow(data_ext2[im,ymin:ymax,xmin:xmax],extent=[xmin,xmax,ymin,ymax],vmin=vmin,vmax=vmax)
                ax = fig.add_subplot(1,4,4)
                ax.set_title('New Dust='+label[1])
                plt.imshow(data_dust[im,ymin:ymax,xmin:xmax],extent=[xmin,xmax,ymin,ymax],vmin=vmin,vmax=vmax)
                plt.show()
                print(xlabel)
                print(" ")
            print('')
            print('Summary amplifier ',im+1,' number of light default(s) (trans>0.6)',defect_list[0],' number of strong default(s)  (trans<0.6)',defect_list[1])
            print('For each dust spot : nb pixels & <occultation> ==> ', spot_list[im])
            print('')
            print('')
    #
    number=0
    ocult=0.
    nb_spot=0
    for im in range(16) : 
        for spot in spot_list[im] :
            nb_spot+=1
            number+=spot[4]
            ocult+=spot[5]*spot[4]
    print (' ')
    print ('ccd = ',ccd,', nb total dust spots ',nb_spot,' for a total of ',number,' pixels below thershold (occultation: .9-.85 ) with a  <transparency> =%4.2f' % (ocult/number))
    print (' ')
    return (spot_list,dust_image)

The following box , does the job :
   - it loops on the # raft , identify the sensor included in the raft and collect TS3 and vendor data for it 
   - for each sensor it loops on its amplifiers and look for new "dust spots" and provide a set of information and a plot for each dust spots . 

In [ ]:
#
for raft in raft_list : 
    # automated initialisation from configuration box 
    label_ref=['Vendor','TS3',raft['Dust1_lab']+'('+raft['Dust1_run']+')',raft['Dust2_lab']+'('+raft['Dust2_run']+')']
    data_sel=np.zeros((4),dtype=int)
    label=[]
    for i in range(4):
        label.append(label_ref[data_id[i]])
        data_sel[data_id[i]]=i
    print("=================================================================================================")
    print("=================================================================================================")
    print('Processing of raft : ',raft)
    # get BNL_raft_run 
    if raft['Dust1_lab']=='BNL' :
        rtm_bnl=glob.glob(BNL_RAFT_ROOT+raft['RTM']+'/'+raft['Dust1_run']+'/cte_raft/*/*/S*/*_superflat_high.fits')
    else :
        rtm_bnl=glob.glob(SLAC_RAFT_ROOT+raft['RTM']+'/'+raft['Dust1_run']+'/cte_raft/*/*/S*/*_superflat_high.fits')
    rtm_bnl.sort()
    # get SLAC_raft_run 
    if raft['Dust2_lab']=='BNL' :
        rtm_slac=glob.glob(BNL_RAFT_ROOT+raft['RTM']+'/'+raft['Dust2_run']+'/cte_raft/*/*/S*/*_superflat_high.fits')
    else :
        rtm_slac=glob.glob(SLAC_RAFT_ROOT+raft['RTM']+'/'+raft['Dust2_run']+'/cte_raft/*/*/S*/*_superflat_high.fits')
    rtm_slac.sort()
    # extract the CCD list for this raft 
    ccd_list=[]
    ts3_list=[]
    vendor_list=[]
    run=0
    # as all raft have been tested at BNL we can get the sensor list from there ... May be issue in this 
    # logic after the last re-furbisshing where some new CCD have been put in raft ... 
    if len(rtm_slac)==0 :
        rtm_for_sensor=rtm_bnl
    else :
        rtm_for_sensor=rtm_slac
    for superflat in rtm_for_sensor[-9:] :
        tree=superflat.split('/')
        if run != 0 and run != tree[-6] :
            print ('Error : # run found for what should be a single raft run ',run,'#',tree[-6],'(all ccd superflats:',rtm_bn[-9:],')')
            raise 
        else : 
            run=tree[-6]
        ccd_list.append(tree[-1][:-20])
    #
    spot_all=[]
    # get ts3 and vendor data for these CCD
    for ccd in ccd_list : 
        dir=BNL_TS3_ROOT+'/'+ccd+'/*/cte/*/*/'+ccd+'_superflat_high.fits'
        ts3=glob.glob(BNL_TS3_ROOT+'/'+ccd+'/*/cte/*/*/'+ccd+'_superflat_high.fits')
        ts3.sort()
        ts3_list.append(ts3)
        vendor=glob.glob(VENDOR_ROOT+'/'+ccd+'/*/cte_offline/*/*/'+ccd+'_superflat_high.fits')
        if len(vendor)==0 : vendor=glob.glob(VENDOR_ROOT+'/'+ccd+'/cte_offline/*/*/'+ccd+'_superflat_high.fits')
        vendor.sort()
        vendor_list.append(vendor)
    #
    im_dust=np.zeros((4),dtype=np.object_)
    #
    for iccd in range(len(ccd_list)) :
        print("=================================================================================================")
        print('Processing of ccd : ',ccd_list[iccd])
        # overscan correct the image 
        if len(rtm_bnl)>0 :
            im_dust[data_sel[2]]=cor_image(rtm_bnl[-9+iccd],label[data_sel[2]])
        elif data_id[0]==2 or data_id[1]==2 :
            print('No Raft BNL data data for ',ccd_list[iccd],'???? Software bug , this is not possible')
            break
        else : 
            im_dust[data_sel[2]]=[]
        if len(rtm_slac)>0 :
            im_dust[data_sel[3]]=cor_image(rtm_slac[-9+iccd],label[data_sel[3]])
        elif data_id[0]==3 or data_id[1]==3 :
            print('No Raft SLAC data data for ',ccd_list[iccd])
            continue
        else : 
            im_dust[data_sel[3]]=[]
        if len(ts3_list[iccd])>0 :
            im_dust[data_sel[1]]=cor_image(ts3_list[iccd][-1],label[data_sel[1]])
        elif data_id[0]==1 or data_id[1]==1 :
            print('No TS3 BNL data data for ',ccd_list[iccd])
            continue
        else :
            im_dust[data_sel[1]]=[]
        if len(vendor_list[iccd])>0 :
            im_dust[data_sel[0]]=cor_image(vendor_list[iccd][-1],label[data_sel[0]])
        elif data_id[0]==0 or data_id[1]==0 :
            print('No vendor data for ',ccd_list[iccd],'???? this should not be possible ')
            break
        else :
            im_dust[data_sel[0]]=[]
        # look for dust 
        spot_all.append((ccd_list[iccd],found_and_plot_dust_spot(ccd_list[iccd],im_dust[0],im_dust[1],im_dust[2],im_dust[3],label,cut=0.15)))
# save the spots for a future usage
# dump in a pickle file
    pname="dust_spot_%s.pkl" % (raft['RTM'])
    # open
    pfile=open(pname,'wb')
    pickle.dump(spot_all,pfile)


======== Example on how to read the pickle file produced with all the identified spots ===

In [ ]:
pname="dust_spot_%s.pkl" % (raft['RTM'])
# open
pfile=open(pname,'rb')
all_spot=pickle.load(pfile) 
# loop on sensor
for spot_ccd in all_spot :
    # name of the ccd
    ccd_name=spot_ccd[0]
    # spot_list summary for each spot
    spot_list=spot_ccd[1][0]
    # normalized image from the "ref" +/-10 pixels arround the spot 
    dust_image=spot_ccd[1][1]
    print('CCD=',ccd_name)
    for i in range(16):
        # loop on amplifier for this CCD
        print('Amplifier=',i)
        nb_spot=len(spot_list[i])
        for j in range(nb_spot) :
            spot_cur=spot_list[i][j]
            ymin_image=dust_image[i][j][0]
            xmin_image=dust_image[i][j][1]
            image=dust_image[i][j][2]
            print('ymin,ymax,xmin,xmax,nb_pixel below treshold,mean tranparency =',  spot_cur,'ymin,xmin image,len_image',ymin_image,xmin_image,len(image))
